# FOSI and hist processing vs. altimeter 1993-2018

In [6]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import scipy.stats
from scipy import signal
from functools import partial
import glob
import dask
import os
import nc_time_axis
import matplotlib.pyplot as plt
#from geocat.viz import util as gvutil
import cartopy
import xesmf as xe
import xskillscore as xs
#cartopy.config['pre_existing_data_dir']='/ihesp/shared/cartopy_features'
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import matplotlib.ticker as mticker
%matplotlib inline

from mypyutils import calendar_utils as cal
from mypyutils import stat_utils as stat
from mypyutils import mapplot_utils as maps
from mypyutils import colorbar_utils as cbars
from mypyutils import io_utils as io
from mypyutils import regrid_utils as regrid

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def detrend_linear2(da, dim):
    """ linear detrend DataArray along the axis dim """
    params = da.polyfit(dim=dim, deg=1)
    fit = xr.polyval(da[dim], params.polyfit_coefficients)
    da = da-fit
    return da, params.polyfit_coefficients[0,:,:]

def detrend_linear_tg(da, dim):
    """ linear detrend DataArray along the axis dim """
    params = da.polyfit(dim=dim, deg=1)
    fit = xr.polyval(da[dim], params.polyfit_coefficients)
    da = da-fit
    return da, params.polyfit_coefficients[0,:]
    
def detrend_deseason_tg(da): #order/fs/low pass filter unused so far
    [fosi_det, fosi_trend]=detrend_linear_tg(da, 'time')
    fosi_trend=fosi_trend * 1e9  * 60  * 60  * 24 * 365
    fosi_mean_sea=fosi_det.groupby("time.month").mean("time")
    fosi_desea=fosi_det.groupby("time.month")-fosi_mean_sea  
    return fosi_desea.drop_vars('month'), fosi_trend
    
def detrend_deseason(da): #order/fs/low pass filter unused so far
    [fosi_det, fosi_trend]=detrend_linear2(da, 'time')
    fosi_trend=fosi_trend * 1e9  * 60  * 60  * 24 * 365
    fosi_mean_sea=fosi_det.groupby("time.month").mean("time")
    fosi_desea=fosi_det.groupby("time.month")-fosi_mean_sea
    return fosi_desea.drop_vars('month'), fosi_trend

In [8]:
import dask
from dask.distributed import wait
dask.__version__

'2023.7.0'

## Create Dask Cluster

In [36]:
# Close out Dask Cluster and release workers:
# NOTE:  only run this cell to terminate Dask Cluster!
client.shutdown()
!rm dask-worker*

In [10]:
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='30GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=30GB',
        account='UARH0002',
        walltime='02:00:00',
        interface='ext',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status',
        'array.slicing.split_large_chunks': True
    })
    client = Client(cluster)
    return cluster, client

In [11]:
cluster, client = get_ClusterClient()
cluster.scale(20) 

/glade/work/clittle/conda-envs/mypythonenv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42559 instead
  warnings.warn(


In [12]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/clittle/proxy/42559/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/clittle/proxy/42559/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.178:46063,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/clittle/proxy/42559/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Altimetry input

In [13]:
datadir = '/glade/work/clittle/altimetry/MEASURES/monthly/'
alt=xr.open_mfdataset(datadir+'monthly*.nc')
alt=alt*100

In [14]:
# alt=alt.rename_dims({'Latitude': 'nlat','Longitude': 'nlon'})
alt=alt.rename({'Time': 'time','Longitude':'lon','Lon_bounds':'lon_b','Latitude':'lat','Lat_bounds':'lat_b'})
# alt=alt.groupby('time.year').mean('time').rename({'year':'time'}) 
alt = alt.drop(['lon_b','lat_b'])
ds_alt=alt.sel(time=slice('1992-01-01', '2019-01-01'))#,Longitude=slice(xlo,xhi),Latitude=slice(ylo,yhi))
# ds_alt=ds_alt.drop_indexes({'time','lon','lat'})

/glade/derecho/scratch/clittle/tmp/ipykernel_89199/2337161318.py:4: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  alt = alt.drop(['lon_b','lat_b'])


In [15]:
gmsl=xr.open_mfdataset('../USEC_FOSI_rez_comp/gmsl_mo.nc')
gmsl=gmsl.gmsl_variation/10

In [16]:
da=ds_alt.SLA-gmsl

In [17]:
xlo=1550; xhi=2200; ylo=550; yhi=900;
da=da.isel(lon=slice(xlo,xhi),lat=slice(ylo,yhi))#.sel(time=slice('1993-01-01','2019-1-1'))
# # # tt.ssh[0,:,:].plot()
# alt_ec.to_netcdf('/glade/work/clittle/p2679/alt_native_ec_big.nc',mode='w')

In [31]:
ds=xr.open_dataset('/glade/work/yeager/datashare/ALPACA/g.e21.GIAF_JRA_v1p5.TL319_t13.001.pop.h.000101_006612.SSH.nc')
                   # , parallel=True)
ds=ds.chunk(40, 100, 100)
# ds=xr.open_mfdataset('/glade/work/yeager/datashare/ALPACA/g.e21.GIAF_JRA_v1p5.TL319_g17.004.pop.h.000101_006612.SSH.nc')

In [32]:
ds = cal.time_set_midmonth(ds, "time") #LR
ds=ds.where(ds.SSH < 1e5)
da=ds.SSH#.isel(nlon=slice(xlo,xhi),nlat=slice(ylo,yhi))
ylo=1400; yhi=1800; xlo=200; xhi=700; #HR2 EC SMALL
da=da.isel(time=slice(0,792-24))#.isel(nlon=slice(xlo,xhi),nlat=slice(ylo,yhi))

In [ ]:
import momlevel
import pandas as pd
tgcsvin="./tgs_in2.csv" #need to read in from csv for momlevel
tgs_in=pd.read_pickle("./tgs_in2.pkl")
omask=xr.where(np.isnan(da[0,:,:]), 0, 1)
tgs_hr_xr=momlevel.extract_tidegauge(da, da.TLONG, da.TLAT, mask=omask, csv=tgcsvin)
tgs_hr_xr=tgs_hr_xr.to_array("tgindex", name="ssh").to_dataset().assign_coords(
    tgindex=tgs_in.name.to_list()
)
hr_tg=tgs_hr_xr.ssh.load()

In [ ]:
[hrdesea, hrtrend]=detrend_deseason_tg(hr_tg)
hrdesea=hrdesea.groupby('time.year').mean('time').rename({'year':'time'})
hrdesea

In [ ]:
# hrdesea
hrdesea=hrdesea.to_dataset(name='ssh')
# hrdesea=hrdesea.to_dataset(name='tauy')
hrdesea.to_netcdf('/glade/work/clittle/p2679/alpaca_hr_tg_ssh_long.nc',mode='w')

In [23]:
hr_tg

<xarray.DataArray 'ssh' (tgindex: 9, time: 768)>
array([[-28.503202, -39.19227 , -32.223183, ..., -18.48232 , -19.031351,
        -24.782007],
       [-31.513905, -41.926132, -34.280956, ..., -19.155954, -22.649931,
        -26.98476 ],
       [-31.717646, -49.993706, -34.093834, ..., -19.632978, -18.521347,
        -27.774061],
       ...,
       [-56.94971 , -61.611748, -49.411148, ..., -46.893402, -55.93115 ,
        -63.258835],
       [-55.547314, -60.81912 , -44.623623, ..., -47.070786, -60.37947 ,
        -67.78336 ],
       [-56.09606 , -60.80475 , -46.81963 , ..., -47.753986, -60.41664 ,
        -65.52117 ]], dtype=float32)
Coordinates:
  * time     (time) object 0001-01-15 00:00:00 ... 0064-12-15 00:00:00
  * tgindex  (tgindex) <U3 '112' '395' '234' '396' '299' '366' '12' '351' '235'